# Análise do risco de inadimplência dos mutuários

Seu projeto é preparar um relatório para a divisão de empréstimos de um banco. Você precisará descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

Seu relatório será considerado ao construir a **pontuação de crédito** de um cliente em potencial. A **pontuação de crédito** é usada para avaliar a capacidade de um devedor em potencial de pagar seu empréstimo.


### Propósito do Projeto
Este projeto tem como objetivo identificar possíveis relações entre informações qualitativas de nossos clientes e sua respectiva capacidade de pagamento de empréstimos.

 - Propósito 1: Obter um score de crédito o mais acurado possível para nossos clientes.
 - Propósito 2: Entender as relações entre características de nossos clientes, e seu comportamento de pagamento de empréstimos.
 - Propósito 3: Com os propósitos acima atingidos, o terceiro propósito trata-se sobre melhorar nossa tomada de decisão quando se tratando do fornecimento de empréstimos para nossos clientes, dadas as informações que temos sobre eles e sobre a relação dessas informações com o comportamento de pagamento de empréstimos destes.
 
### Hipóteses para testar
Hipótese 1: O **estado civil** dos clientes tem impacto sobre seu comportamento de pagamento de empréstimos.

Hipótese 2: O **número de filhos** dos clientes tem impacto sobre seu comportamento de pagamento de empréstimos. 



## Abra o arquivo de dados e veja a informação geral.


Importando livraria 'pandas' e salvando em df, caminho do arquivo /datasets/credit_scoring_eng.csv

In [2]:
# Carregando todas as bibliotecas
import pandas as pd


In [3]:
# Carregue os dados
df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Tarefa 1. Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo


Obtendo informações gerais do dataset através do .shape

In [4]:
# Vamos ver quantas linhas e colunas nosso conjunto de dados tem
print(df.shape)
df_inicial = 21525 #para comparar com o df depois das alterações

(21525, 12)


In [5]:
# vamos imprimir as primeiras N linhas
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family



### Observações Iniciais

A primeira observação é que a coluna days_employed parece conter diversos erros:
1. Dias Negativos
2. Dias de experiência que quando divididos por 365 (para ter exp. em anos) chegam a ser maior que a própria idade do cliente
3. Formato float

Os dois primeiros pontos merecem ser investigados com mais atenção para entender o que pode ter causado esses erros. 

A segunda observação é que os dados da coluna ´education´ não estão padronizados; temos strings todas minúsculas, algumas com maiúsculas e minúsculas e outras completamente maiúsculas. Devemos transformar tudo em lowercase para melhor análise, e para eventualmente encontrar duplicatas implícitas onde a escrita da string está errada mesmo, não sendo uma questão de casing.


In [6]:
# Obter informações sobre dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB




Terceira observação: temos valores ausentes para as colunas ´days_employed´ e ´total_income´, e que o número de ausentes em ambas as colunas é o mesmo. Uma hipótese para isso é que se o cliente não tem emprego, ele não terá income e nem dias de emprego - podemos testar isso observando se as linhas com valores ausentes em income são as mesmas linhas com valores ausentes em dias de emprego.

Outro ponto é que days_employed está como float, interessante investigar porque, em vez de int.

In [7]:
# Vejamos a tabela filtrada com valores ausentes na primeira coluna com dados ausentes
print(df.isna().sum())
df.loc[df['days_employed'].isna()]

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate



Os valores ausentes entre days_employed e total_income batem em número, isto é, parecem simétricos. Resta analisar se são as mesmas linhas que tem valores ausentes em ambas as colunas, e entender se possuem relação com alguma outra informação.

In [8]:
# Vamos aplicar várias condições para filtrar dados e observar o número de linhas na tabela filtrada.

nan_slice = df.loc[(df['total_income'].isna() == True) & (df['days_employed'].isna() == True)]
display(nan_slice)

nan_percent = (len(nan_slice))/(len(df))*100

print(nan_percent)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


10.099883855981417


**Conclusão intermediária**



Realmente, sempre que days_employed está com valor ausente, total_income também está. Isso é visto através do número de linhas (2174) - o mesmo valor para ambos, filtrados na condição de ambas as colunas possuírem esse valor ausente na linha. Isso quer dizer que provavelmente existe alguma relação entre essas duas colunas, para ambas não terem sido preenchidas simultaneamente nesses casos. Resta entender porque, e se tem relação com alguma outra métrica.


Os valores **ausentes representam cerca de 10% do total** dos dados. Podemos tentar preenchê-los. Se não tivermos valores outliers que distorcem a média, usamos a média. Caso sim, podemos optar pela mediana. O ponto principal é saber quais filtros aplicar para ter a média ou mediana mais adequada para preencher esses valores ausentes. Para isso, vamos investigar um pouco mais as outras características das linhas que tem esses valores ausentes.



Os próximos passos consistem em:
1. Verificar relações entre outros dados e os valores ausentes.
2. Verificar possibilidade de preencher os valores ausentes.





In [9]:
# Vamos investigar clientes que não possuem dados sobre as características identificadas e a coluna com os valores ausentes
nan_slice = df.loc[(df['total_income'].isna() == True) & (df['days_employed'].isna() == True)]

In [10]:
# Verificar a distribuição
print(nan_slice['income_type'].value_counts(normalize = True)*100)
print()
print(nan_slice['education_id'].value_counts(normalize = True)*100)
print()
print(nan_slice['debt'].value_counts(normalize = True)*100)


employee         50.827967
business         23.367065
retiree          18.997240
civil servant     6.761730
entrepreneur      0.045998
Name: income_type, dtype: float64

1    70.837167
0    25.022999
2     3.173873
3     0.965961
Name: education_id, dtype: float64

0    92.180313
1     7.819687
Name: debt, dtype: float64




A maior parte do income_type das linhas com valores ausentes é da categoria employee, seguida por business. Isso descarta a possibilidade de não terem preenchido esses dados por estarem desempregados. 

**Possíveis motivos para valores ausentes nos dados**


A princípio, não vejo nenhum destaque, ou seja, nenhum dos outros dados apresenta quase 100% de ocorrência nas linhas que tem valores ausentes. Chama um pouco de atenção o education_id, 70% com id de 1 para as linhas de valores ausentes. Acho que os valores podem estar em falta por 2 motivos em particular:
1. Algum erro na base de dados, ou no preenchimento dos dados originalmente.
2. Participantes simplesmente não quiseram informar esses dados.



In [11]:
# Verificar a distribuição em todo o conjunto de dados
print(df['income_type'].value_counts(normalize = True)*100)
print()
print(df['education_id'].value_counts(normalize = True)*100)
print()
print(df['debt'].value_counts(normalize = True)*100)


employee                       51.656214
business                       23.623693
retiree                        17.914053
civil servant                   6.778165
unemployed                      0.009292
entrepreneur                    0.009292
paternity / maternity leave     0.004646
student                         0.004646
Name: income_type, dtype: float64

1    70.768873
0    24.436702
2     3.456446
3     1.310105
4     0.027875
Name: education_id, dtype: float64

0    91.911731
1     8.088269
Name: debt, dtype: float64


**Conclusão intermediária**



A distribuição do conjunto de dados original é muito similar a distribuição do slice de valores ausentes, sem padrões claros. Isso significa que há grande probabilidade desses valores serem 'MAR' ou 'MCAR', isto é, são valores ausentes aleatórios, e logo podem ser preenchidos com valores padrão, nesses casos, a média ou mediana - vamos investigar qual a mais adequada baseado na presença ou não de outliers que distorcem a média.



**Conclusões**



Em relação aos valores ausentes, sem padrões identificados - isto é, a distribuição dos valores para o slice do df de valores ausentes é bem similar a distribuição do df completo, o que nos leva a crer que os valores ausentes são 'MAR' ou 'MCAR'.


Para abordar os valores ausentes, vamos preenchê-los com valores padrão. Isto é, média ou mediana do conjunto de dados. Vejo duas maneiras de fazer isso - a primeira seria simplesmente preencher os valores com a média/mediana geral, sem qualquer distinção entre o resto das categorias/dados. A segunda maneira, seria uma abordagem mais precisa - vamos definir ranges de idade `dob_years` e `days_employed`, e fazer filtragens consecutivas para as colunas `education_id, family_status_id,	gender,	income_type`. Vamos então pegar a média de `days_employed` e `total_income` desses slices filtrados, e preencher os valores ausentes baseado na combinação desses filtros em cada linha de valor ausente. Para isso, podemos pegar os slices de valores ausentes que se encaixam nesses filtros, e preenchê-los.  


Sobre os outros problemas de dados que encontramos: 

1. **days_employed** está negativo e em formato float, com alguns valores que quando convertidos para anos, são maiores que a idade da pessoa. Vamos corrigir isso - procurar se tem algum valor específico que fez com que convertesse a coluna em float, converter os valores em int, e torná-los positivos. 

2. **education** precisa ser padronizada - vamos jogar tudo para lowercase, e padronizar a escrita.

3. Precisamos verificar duplicados primeiro, pois estes podem afetar a média que poderá ser usada para preencher os valores ausentes.

Outro ponto que gostaria de analisar, é se temos valores que não fazem sentido no dob_years, pois teoricamente não devemos ter nada aqui abaixo de 18 anos. Por exemplo, se for 0, sabemos que foi algum erro, e portanto devemos preenchê-los com valores padrão também - penso em preencher baseado na média por days_employed.

In [14]:
print(df['dob_years'].describe())
print()

below_18_filter = df.loc[df['dob_years'] <= 18]
below_18 = len(below_18_filter)
print(f'Linhas com idade abaixo de 18 = {below_18}, substituir por média ou mediana')


count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Linhas com idade abaixo de 18 = 101, substituir por média ou mediana


Vemos que temos idade de 0 no dob_years, o que não faz sentido para essa base de dados. Portanto, vamos substituir os valores abaixo de 18 com uma média baseada no days_employed. Mas antes, precisamos popular o days_employed.

## Transformação de dados
Vamos verificar valores estranhos em cada coluna da df, começando pela coluna 'education'.


In [15]:
# Vamos ver todos os valores na coluna de educação para verificar se e quais grafias precisarão ser corrigidas
sorted(df['education'].unique())

["BACHELOR'S DEGREE",
 "Bachelor's Degree",
 'GRADUATE DEGREE',
 'Graduate Degree',
 'PRIMARY EDUCATION',
 'Primary Education',
 'SECONDARY EDUCATION',
 'SOME COLLEGE',
 'Secondary Education',
 'Some College',
 "bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

In [16]:
# Corrija os registros, se necessário
df['education'] = df['education'].str.lower()

In [17]:
# Verificando todos os valores na coluna para ter certeza de que os corrigimos
print(sorted(df['education'].unique()))
#Nenhum registro precisa ser corrigido. Passar os dados para lowercase resolveu o problema.

["bachelor's degree", 'graduate degree', 'primary education', 'secondary education', 'some college']


Verificando dados para coluna 'children'

In [18]:
# Vamos ver a distribuição de valores na coluna `children`
print(df['children'].value_counts())
print((len(df.loc[df['children'] == -1])/len(df['children']))*100)


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
0.2183507549361208



Há algo estranho com os dados - tem 47 linhas/pessoas (0.21% dos dados, parcela ínfima) com número de crianças negativo. Não faz sentido alguém ter menos uma criança - faz sentido ter uma criança; o que me leva a crer que talvez tenha sido algum erro de input. Vou tirar o negativo e assumir 1 criança apenas. Outra coisa que chama atenção, é 76 pessoas com 20 crianças. O que me leva a crer que isso também é um erro de input e que não temos ninguém com 19, 18 ... crianças, portanto, imagino que também tenha sido um erro de input onde a pessoa colocou um 0 depois do 2, e na verdade são 2 crianças em vez de 20.

In [19]:
# [corrija os dados com base na sua decisão]

df.loc[df['children'] == -1] = 1
df.loc[df['children'] == 20] = 2




In [20]:
# Verificar a coluna `children` novamente para ter certeza de que está tudo corrigido
df['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Verificando 'days_employed'

In [21]:
# Encontre dados problemáticos em `days_employed`, se existirem, e calcule a porcentagem
print(df['days_employed'].value_counts())
print()
print((len(df.loc[df['days_employed'] < 0])/len(df))*100) #para porcentagem de dados inadequados
print()
print(df['days_employed'].describe())


 2.000000         76
 1.000000         47
-4399.664923       1
 361900.083341     1
-4122.460569       1
                  ..
-201.643573        1
-7120.517564       1
-2146.884040       1
-881.454684        1
-3382.113891       1
Name: days_employed, Length: 19242, dtype: int64

73.44483159117306

count     19363.000000
mean      62758.619571
std      140570.184570
min      -18388.949901
25%       -2732.026542
50%       -1191.592257
75%        -275.419809
max      401755.400475
Name: days_employed, dtype: float64



A maioria dos dados está incorreta - 73% com valores de days_employed negativo. A solução mais óbvia é converter esses dados para o positivo, multiplicando todos eles por -1, ou usando .abs(). Outra correção seria transformar para int, não precisamos desses dados em formato float (é improvável que a pessoa colocaria que trabalhou 880 dias e meio (880.5), o que me leva a crer que podemos ter alguns dados ali que fizeram o python acreditar que essa coluna era float, em vez de int, mas isso fica para um próximo momento.

Outra observação é que parece que temos dados no days_employed que são maiores que a idade (dob_years) das pessoas. Como não podemos inferir o valor a partir do que foi inputado ainda, pretendo transformá-los em NANs para preencher depois com média ou mediana de alguma seleção de filtros. Importante ressaltar que days_employed deveria no mínimo ser menor que a idade (dob_years) da pessoa, menos cerca de 18 anos, caso contrário, seria como se a pessoa trabalhasse já desde seus 0 anos de idade, o que não faz sentido. Isso também será levado em consideração na correção dos dados a seguir.


In [22]:
# Aborde os valores problemáticos, se existirem

#df.loc[df['days_employed'] < 0, 'days_employed'] *= -1
#vi que também era possível ter utilizado o método .abs() para converter os valores da coluna para positivo.

df.loc[:, 'days_employed'] = df.loc[:, 'days_employed'].abs()

In [23]:
#Criando uma formula para identificar quais linhas tem valores de days_employed maiores que dob_years
def days_years(row):
    days_to_years = row['days_employed']/365
    years = row['dob_years'] - 18
    
    if days_to_years > years:
        return 1
    return 0

In [24]:
#Criando coluna para identificar as linhas com days_employed maiores que dob_years. Se sim, 1, se não, 0.
df['days_emp_vs_age'] = df.apply(days_years, axis = 1)

#Transformando os dados de 'days_employed' para NaN onde 'days_emp_vs_age' for 1:
df.loc[df['days_emp_vs_age'] == 1, 'days_employed'] = None

#Checando quantas linhas estão nessas condições
print(len(df.loc[df['days_emp_vs_age'] == 1]))

4031


In [25]:
# Verifique o resultado - certifique-se de que está corrigido
df['days_employed'].describe()


count    15332.000000
mean      2257.807084
std       2164.581235
min         24.141633
25%        735.402446
50%       1594.192772
75%       3063.870521
max      16119.687737
Name: days_employed, dtype: float64

Verificando 'dob_years'

In [26]:
# Verifique o `dob_years` para valores suspeitos e conte a porcentagem
print(df['dob_years'].value_counts())
print()
print(df['dob_years'].describe())
print()
print((below_18/len(df['dob_years']))*100)

35    614
41    603
40    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
44    543
29    543
30    536
48    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
57    457
53    457
51    446
55    441
59    441
26    406
60    376
25    356
61    353
62    351
63    268
24    263
64    263
23    252
65    194
66    183
22    183
67    167
21    110
0     100
68     99
69     83
2      76
70     65
71     58
20     51
1      47
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

count    21525.000000
mean        43.062021
std         12.930644
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

0.4692218350754936




Uma parcela ínfima dos dados (1.03%) apresentou dob_years menor que 18 anos (teoricamente, pessoas com menos de 18 anos não trabalham, a não ser talvez em regime de estágio). Além disso, seria interessante verificar mais a frente se faz sentido a relação da coluna dob_years com a de days_employed, mas neste momento, vou optar por eliminar as linhas que tem dados com dob_years menores que 18, pois não faz sentido, e também porque se trata de uma parcela ínfima do df total, ou seja, o trabalho para arrumar esses dados talvez custe muito pelo retorno de ter mais 1.03% dos dados.


In [27]:
# Resolva os problemas na coluna `dob_years`, se existirem

df[df['dob_years'] <= 18] = df.drop(df[df['dob_years'] <= 18].index, inplace=True)



In [28]:
# Verifique o resultado - certifique-se de que está corrigido
len(df[df['dob_years'] <= 18])

0

Verificando 'family_status'

In [29]:
# Vamos ver os valores da coluna
df['family_status'].value_counts()


married              12254
civil partnership     4139
unmarried             2783
divorced              1179
widow / widower        947
Name: family_status, dtype: int64

Sem problemas encontrados para family_status.

Verificando 'gender'.

In [30]:
# Vamos ver os valores na coluna

df['gender'].value_counts()

F      14083
M       7218
XNA        1
Name: gender, dtype: int64

In [31]:
# Aborde os valores problemáticos, se existirem

#Um valor com XNA - não vai afetar o df, e como não podemos inferir se M ou F, vamos deletar a linha
df.loc[df['gender'] == 'XNA', 'gender'] = df.drop(df.loc[df['gender'] == 'XNA', 'gender'].index, inplace = True)


In [32]:
# Verifique o resultado - certifique-se de que está corrigido
len(df.loc[df['gender'] == 'XNA', 'gender'])


0

Um valor com XNA - como não podemos inferir se M ou F, vamos deletar a linha - uma linha única não vai afetar a análise do df.

Verificando 'income_type'.

In [33]:
# Vamos ver os valores na coluna
df['income_type'].value_counts()

employee                       10996
business                        5033
retiree                         3819
civil servant                   1447
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

Sem valores problemáticos em income_type, a princípio.

Checando duplicados.

In [34]:
# Verificar duplicados
df.duplicated().sum()

71

Temos 71 linhas duplicadas, um número pequeno em relação ao total de dados. Vamos remover essas duplicatas, para que não nos atrapalhe para as próximas etapas de preencher valores ausentes.

In [35]:
# Corrija os duplicados, se existirem

df = df.drop_duplicates().reset_index(drop = True)

In [36]:
# Última verificação se temos duplicados
df.duplicated().sum()

0

In [37]:
#Verifique o tamanho do conjunto de dados que você tem agora após suas primeiras manipulações com ele
print(df.shape)
print(f'Df inicial = {df_inicial}, Df atual = {len(df)}')
print(f'% de redução: {((df_inicial - len(df))/df_inicial)*100}')


(21230, 13)
Df inicial = 21525, Df atual = 21230
% de redução: 1.3704994192799071




As mudanças trouxeram uma redução no conjunto de dados de 1.37%, ou seja, pouco impacto. As mudanças foram as seguintes:
1. Alteramos a coluna 'education' para lowercase.
2. Eliminamos registros com dob_years menor do que 18.
3. Alteramos dados na coluna 'children' que estavam muito fora do razoável e que estavam negativos.
4. Tornamos positivos os valores de 'days_employed' que estavam negativos.
4. Tornamos 'none' os valores onde 'days_employed' estava maior que a idade da pessoa - 18 anos.
5. Em 'gender', excluímos uma linha que estava com valor 'XNA'
6. Removemos duplicatas explícitas.


# Trabalhando com valores ausentes


Porque usar dicionários para alguns valores? Porque dessa forma, consigo identificar a informação que quero apenas com um número índice. Exemplo: em vez de ter que escrever 'bachelor's degree' para filtrar linhas com esse valor na coluna education, posso usar filtrar na coluna education_id o valor 0. É mais rápido e evita erros de digitação.

Vou criar dicionários para educação e para family status.

In [40]:
# Encontre os dicionários

dicionario_educacao = df.loc[:,['education_id', 'education']]
dicionario_educacao['education_id'] = dicionario_educacao['education_id'].astype(int)
dicionario_educacao = dicionario_educacao.drop_duplicates().reset_index(drop = True)
display(dicionario_educacao)

dicionario_family_status = df.loc[:,['family_status_id','family_status']]
dicionario_family_status['family_status_id'] = dicionario_family_status['family_status_id'].astype(int)
dicionario_family_status = dicionario_family_status.drop_duplicates().reset_index(drop = True)
display(dicionario_family_status)

,education_id,education
0,0,bachelor's degree
1,1,secondary education
2,2,some college
3,3,primary education
4,4,graduate degree


,family_status_id,family_status
0,0,married
1,1,civil partnership
2,2,widow / widower
3,3,divorced
4,4,unmarried


### Restaurar valores ausentes em `total_income`


Temos valores ausentes nas colunas 'total_income' e 'days_employed', e é o mesmo número de ausentes em ambas. Foram investigadas, e a princípio parece ser um caso de MAR. Pretendo corrigi-los preenchendo por média ou mediana de um filtro específico que possa afetar essas colunas.

Vamos categorizar a idade, inicialmente, pois pode influenciar no preenchimento dos dados de total_income.


In [41]:
# Vamos escrever uma função que calcule a categoria de idade
def age_group(row):
    age = row['dob_years']
    
    if age < 25:
        return '19-24'
    if 25 <= age < 35:
        return '25-34'
    if 35 <= age < 45:
        return '35-44'
    if 45 <= age < 55:
        return '45-54'
    if 55 <= age <= 65:
        return '55-65'
    if age > 65:
        return '65+'
    

In [42]:
# Teste se a função funciona
print(df['dob_years'].head(5))
print()
print(age_group(df.iloc[0])) #deve retornar 35-44
print(age_group(df.iloc[4])) #deve retornar 45-54

0    42.0
1    36.0
2    33.0
3    32.0
4    53.0
Name: dob_years, dtype: float64

35-44
45-54


In [43]:
# Criar coluna nova com base na função
df['age_group'] = df.apply(age_group, axis = 1)

In [44]:
# Verificar como ficaram os valores na nova coluna
df['age_group'].value_counts()

35-44    5703
25-34    5060
45-54    4840
55-65    4057
19-24     870
65+       700
Name: age_group, dtype: int64

Os fatores que afetam a renda de alguém podem ser, muito provavelmente, a idade, o tipo de renda, o nível de educação e o gênero. Vamos investigar esses fatores mais a frente, para ver como é a relação destes com o total_income.

In [45]:
# Crie uma tabela sem valores ausentes e imprima algumas de suas linhas para garantir que ela fique boa
df_no_na = df.loc[df['total_income'].isna() != True]

display(df_no_na.head(10))
df_no_na.shape
df_no_na.isna().sum() #os valores nulos em days_employed são esperados pois nós os substituimos anteriormente!

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_emp_vs_age,age_group
0,1.0,8437.673028,42.0,bachelor's degree,0.0,married,0.0,F,employee,0.0,40620.102,purchase of the house,0.0,35-44
1,1.0,4024.803754,36.0,secondary education,1.0,married,0.0,F,employee,0.0,17932.802,car purchase,0.0,35-44
2,0.0,NaN,33.0,secondary education,1.0,married,0.0,M,employee,0.0,23341.752,purchase of the house,1.0,25-34
3,3.0,4124.747207,32.0,secondary education,1.0,married,0.0,M,employee,0.0,42820.568,supplementary education,0.0,25-34
4,0.0,NaN,53.0,secondary education,1.0,civil partnership,1.0,F,retiree,0.0,25378.572,to have a wedding,1.0,45-54
5,0.0,926.185831,27.0,bachelor's degree,0.0,civil partnership,1.0,M,business,0.0,40922.170,purchase of the house,0.0,25-34
6,0.0,2879.202052,43.0,bachelor's degree,0.0,married,0.0,F,business,0.0,38484.156,housing transactions,0.0,35-44
7,0.0,152.779569,50.0,secondary education,1.0,married,0.0,M,employee,0.0,21731.829,education,0.0,45-54
8,2.0,NaN,35.0,bachelor's degree,0.0,civil partnership,1.0,F,employee,0.0,15337.093,having a wedding,1.0,35-44
9,0.0,2188.756445,41.0,secondary education,1.0,married,0.0,M,employee,0.0,23108.150,purchase of the house for my family,0.0,35-44


children               0
days_employed       3817
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
days_emp_vs_age        0
age_group              0
dtype: int64

In [46]:
# Veja os valores médios de renda com base nos seus fatores identificados

pt_edu = df_no_na.pivot_table(index = ['age_group', 'gender'],
                    columns = 'education',
                    values = 'total_income',
                    aggfunc = 'mean')
pt_edu

education         bachelor's degree  graduate degree  primary education  \
age_group gender                                                          
19-24     F            23620.715184              NaN       21635.141000   
          M            29094.499105              NaN       26356.526667   
25-34     F            28989.724520              NaN       21113.696824   
          M            36998.975468              NaN       25617.854679   
35-44     F            31575.676235        17822.757       20285.984400   
          M            44784.198541        18551.846       22922.215432   
45-54     F            33416.348527              NaN       21703.293238   
          M            40644.844685        31771.321       26851.186062   
55-65     F            28804.727966        40868.031       18115.126186   
          M            35299.354167        42945.794       19556.078250   
65+       F            26765.414324              NaN       15483.721895   
          M            29623.103097        15800.399       23636.009833   

education         secondary education  some college  
age_group gender                                     
19-24     F              19368.633154  19624.501787  
          M              24223.908995  27871.288315  
25-34     F              21892.461201  26016.419193  
          M              28262.226981  33840.394059  
35-44     F              23617.842308  32682.138722  
          M              30426.369766  36767.788517  
45-54     F              23381.367718  27469.607721  
          M              28837.766428  31949.375630  
55-65     F              22471.324830  27256.598931  
          M              25767.931638  37678.813750  
65+       F              19212.732790  23228.183857  
          M              20628.170451  19946.795000

In [47]:
# Veja os valores medianos de renda com base nos seus fatores identificados
pt_edu_med = df_no_na.pivot_table(index = ['age_group', 'gender'],
                    columns = 'education',
                    values = 'total_income',
                    aggfunc = 'median')

pt_edu_med

education         bachelor's degree  graduate degree  primary education  \
age_group gender                                                          
19-24     F              20248.9425              NaN         21635.1410   
          M              26605.4460              NaN         27119.0240   
25-34     F              25110.4335              NaN         18212.5840   
          M              32489.7725              NaN         22489.8225   
35-44     F              26893.2880        17822.757         18366.6985   
          M              36132.2290        18551.846         21545.9555   
45-54     F              28435.1615              NaN         21215.4590   
          M              34707.5705        31771.321         23302.4850   
55-65     F              25305.4825        40868.031         17183.7920   
          M              30206.5740        42945.794         18251.0060   
65+       F              21979.4380              NaN         13715.3340   
          M              24688.8660        15800.399         21019.6355   

education         secondary education  some college  
age_group gender                                     
19-24     F                17265.4810    19522.4900  
          M                22883.3735    25571.4005  
25-34     F                19812.1790    23136.0940  
          M                25694.8720    29975.7245  
35-44     F                21133.0000    26956.8005  
          M                26605.6140    34384.3045  
45-54     F                20632.2910    22119.5260  
          M                25834.7250    30526.5090  
55-65     F                19609.7190    25351.1100  
          M                23135.5690    32316.3535  
65+       F                16943.4920    28178.9170  
          M                18655.6060    19946.7950

In [48]:
# Veja os valores medianos de renda com base nos seus fatores identificados
pt_inc = df_no_na.pivot_table(index = ['age_group', 'gender'],
                    columns = 'income_type',
                    values = 'total_income',
                    aggfunc = 'mean')
pt_inc

income_type           business  civil servant      employee  entrepreneur  \
age_group gender                                                            
19-24     F       23224.736288   23079.340682  18891.918785           NaN   
          M       29033.187809   24040.661476  24416.748299           NaN   
25-34     F       28010.069102   23842.389471  23328.179959     79866.103   
          M       35937.955142   33165.574685  28698.494706           NaN   
35-44     F       30245.131222   24771.727303  24189.631554           NaN   
          M       40940.625381   36108.251836  30343.066758           NaN   
45-54     F       31152.788204   25933.820795  24079.376424           NaN   
          M       37887.328850   36777.963049  28381.679260           NaN   
55-65     F       30153.748770   25201.307779  25695.637283           NaN   
          M       33909.149490   30905.853414  29113.854702           NaN   
65+       F       32280.223867   31936.574000  24517.505917           NaN   
          M       35453.125556   27687.336500  32188.093100           NaN   

income_type       paternity / maternity leave       retiree   student  \
age_group gender                                                        
19-24     F                               NaN  14298.976000       NaN   
          M                               NaN           NaN  15712.26   
25-34     F                               NaN  24042.723800       NaN   
          M                               NaN  17641.225500       NaN   
35-44     F                          8612.661  22862.408032       NaN   
          M                               NaN  23825.985968       NaN   
45-54     F                               NaN  21863.713895       NaN   
          M                               NaN  28976.185922       NaN   
55-65     F                               NaN  21827.162935       NaN   
          M                               NaN  24119.014180       NaN   
65+       F                               NaN  19305.566810       NaN   
          M                               NaN  20843.732641       NaN   

income_type       unemployed  
age_group gender              
19-24     F              NaN  
          M              NaN  
25-34     F              NaN  
          M         9593.119  
35-44     F              NaN  
          M              NaN  
45-54     F        32435.602  
          M              NaN  
55-65     F              NaN  
          M              NaN  
65+       F              NaN  
          M              NaN

In [49]:
# Veja os valores medianos de renda com base nos seus fatores identificados
pt_inc_med = df_no_na.pivot_table(index = ['age_group', 'gender'],
                    columns = 'income_type',
                    values = 'total_income',
                    aggfunc = 'median')

pt_inc_med

income_type        business  civil servant    employee  entrepreneur  \
age_group gender                                                       
19-24     F       20084.510     19036.1785  17290.1590           NaN   
          M       25939.727     24617.5440  23088.3290           NaN   
25-34     F       24721.904     21291.3050  20723.5700     79866.103   
          M       31726.813     31948.6970  25792.8655           NaN   
35-44     F       26204.964     21580.9365  21390.6900           NaN   
          M       32234.711     30027.2085  26679.4290           NaN   
45-54     F       27154.006     22672.9320  20935.1405           NaN   
          M       33594.343     29002.9830  25712.8155           NaN   
55-65     F       26255.729     23052.4660  22375.3670           NaN   
          M       30462.864     26470.1080  25510.4080           NaN   
65+       F       29299.764     26089.6870  23929.9040           NaN   
          M       32488.958     27687.3365  33314.1415           NaN   

income_type       paternity / maternity leave     retiree   student  \
age_group gender                                                      
19-24     F                               NaN  14298.9760       NaN   
          M                               NaN         NaN  15712.26   
25-34     F                               NaN  19546.0750       NaN   
          M                               NaN  15378.5635       NaN   
35-44     F                          8612.661  18495.7740       NaN   
          M                               NaN  22383.7480       NaN   
45-54     F                               NaN  18692.6160       NaN   
          M                               NaN  26126.8345       NaN   
55-65     F                               NaN  19108.6555       NaN   
          M                               NaN  20769.7875       NaN   
65+       F                               NaN  16642.2935       NaN   
          M                               NaN  18233.8180       NaN   

income_type       unemployed  
age_group gender              
19-24     F              NaN  
          M              NaN  
25-34     F              NaN  
          M         9593.119  
35-44     F              NaN  
          M              NaN  
45-54     F        32435.602  
          M              NaN  
55-65     F              NaN  
          M              NaN  
65+       F              NaN  
          M              NaN

In [50]:
pt_no_age = df_no_na.pivot_table(index = ['education', 'income_type'],
                    columns = 'gender',
                    values = 'total_income',
                    aggfunc = 'median')
pt_no_age

gender                                                    F           M
education           income_type                                        
bachelor's degree   business                     30382.9540  37290.3815
                    civil servant                25274.2540  34086.6370
                    employee                     24609.7600  30977.4250
                    entrepreneur                 79866.1030         NaN
                    retiree                      22769.1740  25193.1730
                    student                             NaN  15712.2600
                    unemployed                   32435.6020         NaN
graduate degree     civil servant                17822.7570         NaN
                    employee                            NaN  31771.3210
                    retiree                      40868.0310  15800.3990
primary education   business                     21441.9210  24016.1540
                    civil servant                14339.0340  30554.6660
                    employee                     18962.6750  22662.0740
                    retiree                      16188.8905  18099.8720
secondary education business                     23166.6445  29132.6230
                    civil servant                20103.5720  26991.4325
                    employee                     19836.1430  24988.4370
                    paternity / maternity leave   8612.6610         NaN
                    retiree                      18025.4190  20051.0390
                    unemployed                          NaN   9593.1190
some college        business                     27318.1230  30606.7090
                    civil servant                21855.8610  28371.3005
                    employee                     21720.3035  29977.7850
                    retiree                      18654.8760  19946.7950

In [51]:
df_no_na.groupby('income_type')['total_income'].mean().sort_values() 

income_type
paternity / maternity leave     8612.661000
student                        15712.260000
unemployed                     21014.360500
retiree                        21950.722935
employee                       25822.872585
civil servant                  27336.442546
business                       32424.420789
entrepreneur                   79866.103000
Name: total_income, dtype: float64

In [52]:
df_no_na.groupby('education')['total_income'].mean().sort_values()  

education
primary education      21144.882211
secondary education    24594.390815
graduate degree        27960.024667
some college           29028.844227
bachelor's degree      33197.258790
Name: total_income, dtype: float64

In [53]:
df_no_na.groupby('gender')['total_income'].mean().sort_values()  

gender
F    24666.721477
M    30913.888696
Name: total_income, dtype: float64

In [54]:
df_no_na.groupby('age_group')['total_income'].mean().sort_values()  

age_group
65+      20805.696512
19-24    22717.058884
55-65    24572.602380
25-34    27349.887937
45-54    27399.896532
35-44    28733.768065
Name: total_income, dtype: float64



Os fatores que muito provavelmente afetam a renda são nível de educação (mais estudo, mais qualificação, mão de obra mais qualificada tende a obter empregos que exigem mais qualificação e portanto pagam mais, uma vez que a oferta de mão de obra qualificada é menor que a oferta de mão de obra não qualificada), idade (que se traduz no mercado de trabalho como experiência para lidar com situações diversas - quanto maior a idade, teoricamente mais experiência, mais capacidade de lidar com situações diferentes, maior o salário), gênero (porque infelizmente de certa forma pessoas do gênero masculino tendem a ter um salário maior). Tipo de renda também pode ser um fator, com algum tipo tendo média de salários maior do que outro tipo (setor público vs privado, por exemplo).




Observando os dados, parece que educação realmente influencia o total_income: quanto maior o grau de educação, maior a remuneração - com exceção para graduate degree, que é um grau de educação mais alto que bachelor's, mas na média e mediana, ganha menos que o bachelor's degree. Também existem diferenças razoáveis entre Male e Female para o total_income (parece que no geral, Male é maior). Em relação a idade, parece que a remuneração maior está no miolo entre 25 a 54 anos, com os grupos mais jovens e mais velhos recebendo menos. Por último, parece que temos também uma diferença pelo income_type. As médias e medianas diferem, sendo as medianas mais conservadoras (total_income menores). Como estamos lidando com o risco de não pagamento dos clientes, opto pela abordamegem mais conservadora, usar a mediana. Além disso, para a média, temos outliers em entrepeneurs.

Com isso, vou usar a mediana dos valores de total_income filtrados por 'gender', 'income_type', 'age_group', 'education'.


In [55]:
#  Escreva uma função que usaremos para preencher os valores ausentes       
def fill_nan(education, age_group, gender):
    try:
        return pt_edu_med[education][age_group][gender]
    except:
        return 'erro'

In [56]:
# Verifique se funciona
fill_nan("bachelor's degree",'25-34','M')

32489.7725

In [57]:
# Aplique em todas as linhas
df['mediana_edu'] = df.apply(lambda row: fill_nan(row['education'],row['age_group'],row['gender']), axis = 1)

In [58]:
df['total_income'] = df['total_income'].fillna(df['mediana_edu'])

In [59]:
# Verifique se temos algum erro
df[df['total_income'] == 'erro']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_emp_vs_age,age_group,mediana_edu



Sem erros encontrados nessa substituição, mas vamos ver como poderíamos corrigir caso tivessemos algum erro - poderíamos pegar algum valor manualmente de alguma das tabelas pivot, e preencher manualmente com o código abaixo:

In [60]:
# Substituir valores ausentes se houver algum erro
### df['total_income'] = df['total_income'].replace('erro', valor)
df['total_income'].dtype

dtype('float64')

Verificando numero total de dados na coluna 'total_income' vs outras colunas.

In [61]:
# Verificar o número de entradas nas colunas
print(len(df['total_income']) - len(df['gender']))
print(df['total_income'].isna().sum())
#Pegando uma coluna que não tinha valores ausentes e comparando o tamanho
#com uma coluna que tinha valores ausentes mas foram preenchidos
#e também checando pelo método isna.
#Queremos 0s aqui.


0
0


###  Restaurar valores em `days_employed`


Para restaurar valores em days_employed, provavelmente as informações que queremos são idade da pessoa, pois em teoria, quanto maior a idade, mais days_employed ela tem. Vou investigar se há grande diferença em relação ao gênero também, e provavelmente no income_type também veremos diferenças, dadas as diferenças nos regimes de emprego.

In [62]:
# Distribuição de `days_employed` medianos com base nos seus parâmetros identificados
display(df.groupby('age_group')['days_employed'].median())
display(df.groupby('gender')['days_employed'].median())
display(df.groupby('income_type')['days_employed'].median())

pt_age_med = df.pivot_table(index = ['age_group','income_type'],
              columns = 'gender',
              values = 'days_employed',
              aggfunc = 'median')
pt_age_med

age_group
19-24     629.134844
25-34    1244.566920
35-44    1774.559168
45-54    2125.569742
55-65    2408.781174
65+      2669.073965
Name: days_employed, dtype: float64

gender
F    1705.540260
M    1405.438904
Name: days_employed, dtype: float64

income_type
business                       1522.836653
civil servant                  2580.789482
employee                       1533.553144
entrepreneur                    520.848083
paternity / maternity leave    3296.759962
retiree                                NaN
student                         578.751554
unemployed                             NaN
Name: days_employed, dtype: float64

gender                                           F            M
age_group income_type                                          
19-24     business                      625.719683   586.562347
          civil servant                 814.965738   597.273431
          employee                      618.956729   644.614108
          student                              NaN   578.751554
25-34     business                     1242.374544  1060.920020
          civil servant                1610.810769  2210.034897
          employee                     1322.195600  1130.966487
          entrepreneur                  520.848083          NaN
35-44     business                     1678.753163  1685.926717
          civil servant                2907.135105  3551.609375
          employee                     1819.389382  1546.898227
          paternity / maternity leave  3296.759962          NaN
45-54     business                     2062.808634  1950.704784
          civil servant                3559.606407  3124.654739
          employee                     2331.334484  1596.297960
55-65     business                     2381.460763  1907.943812
          civil servant                4177.774233  2967.292449
          employee                     2660.657504  2089.453519
65+       business                     3334.259327  1928.123732
          civil servant                2219.855114  4487.652488
          employee                     3114.180553  2468.049954

In [63]:
# Distribuição de `days_employed` médios com base nos seus parâmetros identificados
display(df.groupby('age_group')['days_employed'].mean())
display(df.groupby('gender')['days_employed'].mean())

pt_age_mean = df.pivot_table(index = ['age_group','income_type'],
              columns = 'gender',
              values = 'days_employed',
              aggfunc = 'mean')
pt_age_mean

age_group
19-24     736.198565
25-34    1504.288005
35-44    2280.699143
45-54    2903.492536
55-65    3461.248141
65+      3986.987101
Name: days_employed, dtype: float64

gender
F    2400.947713
M    2020.595524
Name: days_employed, dtype: float64

gender                                           F            M
age_group income_type                                          
19-24     business                      672.260383   678.779342
          civil servant                 877.930879   807.173666
          employee                      753.794181   764.269507
          student                              NaN   578.751554
25-34     business                     1459.096796  1417.344061
          civil servant                1760.653258  2121.005247
          employee                     1539.330200  1415.431281
          entrepreneur                  520.848083          NaN
35-44     business                     2039.422227  2124.135558
          civil servant                3238.658809  3458.255062
          employee                     2335.630253  2116.878282
          paternity / maternity leave  3296.759962          NaN
45-54     business                     2734.642152  2483.962847
          civil servant                4266.917108  3911.242345
          employee                     3103.280844  2363.792435
55-65     business                     3344.238978  2684.328170
          civil servant                5059.522649  3931.070343
          employee                     3754.997826  2928.350027
65+       business                     3759.304420  3668.857968
          civil servant                4048.053548  4487.652488
          employee                     4458.591641  2774.171406

In [64]:
pt_retiree = df.pivot_table(index = 'age_group',
              values = 'days_employed',
              aggfunc = 'median')
pt_retiree

,days_employed
age_group,
19-24,629.134844
25-34,1244.566920
35-44,1774.559168
45-54,2125.569742
55-65,2408.781174
65+,2669.073965



Vou usar medianas, pois dessa forma evito que valores outliers que podem existir afetem a média e consequentemente os dados no geral. Com a mediana, isso não será um possível problema. Interessante notar também que nenhum retiree tem dados para days_employed (o que faz sentido, já que não estão mais empregados).

In [65]:
# Vamos escrever uma função que calcule médias ou medianas (dependendo da sua decisão) com base no seu parâmetro identificado
def fill_nan_days(gender, age_group, income_type):
    try:
        return pt_age_med[gender][age_group][income_type]
    except:
        return 'erro'

In [66]:
# Verifique se a função funciona
fill_nan_days('M','45-54','business')

1950.7047838200888

In [67]:
# Aplicar função ao income_type
df['mediana_days'] = df.apply(lambda row: fill_nan_days(row['gender'],row['age_group'],row['income_type']), axis = 1)


In [68]:
# Verifique se a função funcionou
df['days_employed'] = df['days_employed'].fillna(df['mediana_days'])

print(df['days_employed'].isna().sum())
df.info()

display(df[df['days_employed'] == 'erro'])

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21230 non-null  float64
 1   days_employed     21230 non-null  object 
 2   dob_years         21230 non-null  float64
 3   education         21230 non-null  object 
 4   education_id      21230 non-null  float64
 5   family_status     21230 non-null  object 
 6   family_status_id  21230 non-null  float64
 7   gender            21230 non-null  object 
 8   income_type       21230 non-null  object 
 9   debt              21230 non-null  float64
 10  total_income      21230 non-null  float64
 11  purpose           21230 non-null  object 
 12  days_emp_vs_age   21230 non-null  float64
 13  age_group         21230 non-null  object 
 14  mediana_edu       21230 non-null  float64
 15  mediana_days      21230 non-null  object 
dtypes: float64(8), object(8)
memory usage:

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_emp_vs_age,age_group,mediana_edu,mediana_days
4,0.0,erro,53.0,secondary education,1.0,civil partnership,1.0,F,retiree,0.0,25378.572,to have a wedding,1.0,45-54,20632.291,erro
12,0.0,erro,65.0,secondary education,1.0,civil partnership,1.0,M,retiree,0.0,23135.569,to have a wedding,0.0,55-65,23135.569,erro
18,0.0,erro,53.0,secondary education,1.0,widow / widower,2.0,F,retiree,0.0,9091.804,buying a second-hand car,1.0,45-54,20632.291,erro
24,1.0,erro,57.0,secondary education,1.0,unmarried,4.0,F,retiree,0.0,46487.558,transactions with commercial real estate,1.0,55-65,19609.719,erro
25,0.0,erro,67.0,secondary education,1.0,married,0.0,M,retiree,0.0,8818.041,buy real estate,1.0,65+,18655.606,erro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21210,0.0,erro,53.0,secondary education,1.0,civil partnership,1.0,M,retiree,0.0,12070.399,to have a wedding,1.0,45-54,25834.725,erro
21213,0.0,erro,62.0,secondary education,1.0,married,0.0,M,retiree,0.0,11622.175,property,1.0,55-65,23135.569,erro
21214,0.0,erro,59.0,bachelor's degree,0.0,married,0.0,M,retiree,0.0,11684.650,real estate transactions,1.0,55-65,30206.574,erro
21223,0.0,erro,59.0,secondary education,1.0,married,0.0,F,retiree,0.0,24618.344,purchase of a car,1.0,55-65,19609.719,erro


In [69]:
len(df[df['days_employed'] == 'erro']) - len(df[df['income_type'] == 'retiree'])
#apenas 3 casos onde days_employed deu 'erro' e income_type 'retiree' não - provavelmente os unemployed.
#ou seja, a maioria dos erros na função são para days_employed de retirees.

3

In [70]:
# Substituir valores ausentes
df['days_employed'] = df['days_employed'].replace('erro',2669)
df['mediana_days'] = df['mediana_days'].replace('erro',2669)
# 2669 é a mediana de days_employed para pessoas 65+, geralmente retirees são mais velhos, portanto optei por esse valor manual.
#estou ciente que estou preenchendo também os income_type unemployed com esse valor

[Quando você achar que terminou com `total_income`, verifique se o número total de valores nesta coluna corresponde ao número de valores em outras.]

In [71]:
# Verifique as entradas em todas as colunas - certifique-se de corrigir todos os valores ausentes
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21230 non-null  float64
 1   days_employed     21230 non-null  float64
 2   dob_years         21230 non-null  float64
 3   education         21230 non-null  object 
 4   education_id      21230 non-null  float64
 5   family_status     21230 non-null  object 
 6   family_status_id  21230 non-null  float64
 7   gender            21230 non-null  object 
 8   income_type       21230 non-null  object 
 9   debt              21230 non-null  float64
 10  total_income      21230 non-null  float64
 11  purpose           21230 non-null  object 
 12  days_emp_vs_age   21230 non-null  float64
 13  age_group         21230 non-null  object 
 14  mediana_edu       21230 non-null  float64
 15  mediana_days      21230 non-null  float64
dtypes: float64(10), object(6)
memory usage: 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
days_emp_vs_age     0
age_group           0
mediana_edu         0
mediana_days        0
dtype: int64

days_employed está como float, e não temos mais nenhum valor ausente.

## Categorização de dados


Para decidir quais dados categorizar, vou usar como critério **quantos valores diferentes tem para cada coluna de dados**. Por exemplo, para education, temos uma número razoável de categorias que nos permite analisar os dados facilmente. Já para total_income, temos muitos valores diferentes, o que significa que para extrair alguma informação relevante disso, devemos categorizá-los, como fizemos com a idade.


In [72]:
# Imprima os valores dos dados selecionados para categorização

# Existe uma correlação entre o nível de renda e do pagamento em dia?
#### Categorizar Total income por low, medium, high
print(df['total_income'].value_counts())

# Existe uma correlação entre o status familiar e o pagamento em dia?
### Categorias já definidas, sem necessidade de mais categorização (dadaos já suficientemetne agrupados)


# Existe uma correlação entre numero de filhos e do pagamento em dia?
### Categorias definidas, sem necessidade de agrupar os dados


# Como a finalidade do crédito afeta a taxa de inadimplência?
### Para Purspose, categorizar entre Carro, Casa, Educação e Casamento
print(df['purpose'].value_counts())




20632.291    260
19609.719    252
21133.000    242
19812.179    162
26605.614    140
            ... 
6264.532       1
27097.085      1
45484.109      1
27715.458      1
41428.916      1
Name: total_income, Length: 19169, dtype: int64
wedding ceremony                            785
having a wedding                            759
to have a wedding                           755
real estate transactions                    669
buy commercial real estate                  655
buying property for renting out             647
transactions with commercial real estate    643
housing transactions                        641
purchase of the house for my family         636
housing                                     635
purchase of the house                       634
property                                    627
construction of own property                626
transactions with my real estate            623
building a property                         619
purchase of my own house                    6

In [73]:
# Verifique os valores exclusivos
print(df['purpose'].value_counts())
print()
print(df['total_income'].value_counts())

wedding ceremony                            785
having a wedding                            759
to have a wedding                           755
real estate transactions                    669
buy commercial real estate                  655
buying property for renting out             647
transactions with commercial real estate    643
housing transactions                        641
purchase of the house for my family         636
housing                                     635
purchase of the house                       634
property                                    627
construction of own property                626
transactions with my real estate            623
building a property                         619
purchase of my own house                    618
building a real estate                      617
buy real estate                             612
housing renovation                          602
buy residential real estate                 599
buying my own car                       


### Dados para categorizar

#### 'purpose'
Para purpose, podemos identificar que o propósito do empréstimo está principalmente relacionado a 4 categorias - Carro, Casa, Educação e Casamento.

#### 'total_income'
Para total income, podemos dividir em 3 categorias - high, medium, low.




In [74]:
# Vamos escrever uma função para categorizar os dados com base em tópicos comunss
def purpose_cat(row):
    purpose = row['purpose']
    
    if 'education' in purpose or 'educated' in purpose or 'university' in purpose:
        return 'education'
    if 'car' in purpose or 'cars' in purpose:
        return 'car'
    if 'wedding' in purpose:
        return 'wedding'
    
    return 'property'

#imagino que temos maneiras mais eficientes de fazer isso, mas como estou pressionado pelo tempo, fiz pela maneira que encontrei primeiro

In [75]:
# Crie uma coluna com as categorias e conte os valores para elas
df['purpose_cat'] = df.apply(purpose_cat, axis = 1)
display(df['purpose_cat'].value_counts())

property     10703
car           4258
education     3970
wedding       2299
Name: purpose_cat, dtype: int64

In [77]:
# Obter estatísticas resumidas para a coluna
df['total_income'].describe()
#Isso é para definir os ranges do total income:
#Low será menor que 25%
#Medium-Low será entre 25% e 50%
#Medium-high será entre 50% e 75%
#High será maior que 75%


count     21230.000000
mean      26485.716716
std       15772.958416
min        3306.762000
25%       17140.513000
50%       23135.569000
75%       31624.461250
max      362496.645000
Name: total_income, dtype: float64


Os agrupamentos de total_income foram feitos de maneira a seguir um padrão identificado pelas % de divisão dos dados presentes nessa coluna.

Definição dos ranges de total income:
Low será menor que 25%
Medium-Low será entre 25% e 50%
Medium-high será entre 50% e 75%
High será maior que 75%
da amostra.

In [78]:
# Criar função para categorização em diferentes grupos numéricos com base em intervalos
def total_income_cat(row):
    total_income = row['total_income']
    if total_income < 17140:
        return 'low'
    if 17140 < total_income < 23135:
        return 'medium-low'
    if 23135 < total_income < 31624:
        return 'medium-high'
    if total_income > 31624:
        return 'high'


In [79]:
# Criar coluna com categorias
df['income_cat'] = df.apply(total_income_cat, axis = 1)

In [80]:
# Conte os valores de cada categoria para ver a distribuição
df['income_cat'].value_counts()


medium-high    5312
high           5308
low            5307
medium-low     5303
Name: income_cat, dtype: int64

## Verificar as Hipóteses


**Existe uma correlação entre numero de filhos e do pagamento em dia?**

In [81]:
#alterando linhas de debt por 'True' e 'False' para poder manipular a pivot_table
df.loc[df['debt'] == 1,'debt'] = 'True'
df.loc[df['debt'] == 0, 'debt'] = 'False'


# Verifique os dados das crianças e do pagamento em dia
pt_child = df.pivot_table(index = 'children',
                      columns = 'debt', 
                      values = 'dob_years',
                      aggfunc = 'count',
                         margins = 'count')

# Calcular a taxa de inadimplência com base no número de filhos

inad_child = (pt_child.loc[:,'True']/pt_child.loc[:,'All'])*100

display(inad_child.sort_values())
inad_child.describe()
#Estou ciente que no método describe, a linha 'All' também está sendo levada em consideração

children
0.0    7.545824
All    8.120584
3.0    8.231707
1.0    9.202838
2.0    9.514468
4.0    9.756098
5.0         NaN
dtype: float64

count    6.000000
mean     8.728587
std      0.884736
min      7.545824
25%      8.148365
50%      8.717273
75%      9.436560
max      9.756098
dtype: float64

**Conclusão**

Parece que não há muita diferença na taxa de inadimplência pelo número de filhos, mas há uma diferença entre quem tem filhos e quem não tem: quem não tem filhos tem uma taxa de inadimplência de cerca de 1.2% menor do que a média e mediana geral.

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [82]:
# Verifique os dados de status da família e do pagamento em dia

pt_family = df.pivot_table(index = 'family_status',
                      columns = 'debt', 
                      values = 'dob_years',
                      aggfunc = 'count',
                         margins = 'count')


# Calcular a taxa padrão com base no status da família

inad_family = (pt_family.loc[:,'True']/pt_family.loc[:,'All'])*100


display(inad_family.sort_values())
inad_family.describe()


family_status
widow / widower      6.553911
divorced             7.124682
married              7.557521
All                  8.120584
civil partnership    9.314202
unmarried            9.784173
dtype: float64

count    6.000000
mean     8.075845
std      1.260442
min      6.553911
25%      7.232892
50%      7.839052
75%      9.015798
max      9.784173
dtype: float64

**Conclusão**



A menor taxa de inadimplência de acordo com o status familiar fica com viúvas(os). A mais alta, com não casados. Divorciados e casados também tem inadimplência menor do que a média geral das categorias, enquanto união estável está acima da média.



**Existe uma correlação entre o nivel de renda e o pagamento em dia?**

In [83]:
# Verifique os dados do nível de renda e do pagamento em dia

pt_income = df.pivot_table(index = 'income_cat',
                      columns = 'debt', 
                      values = 'dob_years',
                      aggfunc = 'count',
                         margins = 'count')

# Calcular a taxa de inadimplência com base no nível de renda

inad_income = (pt_income.loc[:,'True']/pt_income.loc[:,'All'])*100



display(inad_income.sort_values())
inad_income.describe()

income_cat
high           7.008289
low            8.045977
All            8.120584
medium-low     8.485763
medium-high    8.942018
dtype: float64

count    5.000000
mean     8.120526
std      0.716044
min      7.008289
25%      8.045977
50%      8.120584
75%      8.485763
max      8.942018
dtype: float64

**Conclusão**



Pelo que vemos nos dados, a categoria de high income tem a menor taxa de inadimplência, seguido por low income (o que vem com certa surpresa - menor income associado a melhor pagamento de empréstimos?) e a pior categoria para medium high income.

**Como a finalidade do crédito afeta a taxa de inadimplência?**

In [84]:
# Confira os percentuais de inadimplência para cada finalidade de crédito e analise-os

pt_purpose = df.pivot_table(index = 'purpose_cat',
                      columns = 'debt', 
                      values = 'dob_years',
                      aggfunc = 'count',
                         margins = 'count')


inad_purpose = (pt_purpose.loc[:,'True']/pt_purpose.loc[:,'All'])*100


display(inad_purpose.sort_values())
inad_purpose.describe()


purpose_cat
property     7.259647
wedding      7.872988
All          8.120584
education    9.294710
car          9.323626
dtype: float64

count    5.000000
mean     8.374311
std      0.909191
min      7.259647
25%      7.872988
50%      8.120584
75%      9.294710
max      9.323626
dtype: float64

**Conclusão**


Em relação ao propósito do empréstimo, as menores taxas são relacionadas a propriedades/casas, e as maiores, educação e carros.


# Conclusão Geral 

## Resumo do projeto e passos

### Limpeza e pré-processamento
Quando recebemos os dados, lidamos com 3 principais questões iniciais para fazer a limpeza e tratamento destes, para que então possamos usá-los. 

**O primeiro passo** é examinar os dados no geral, ver o tipo de cada coluna, ver se temos valores nulos e o porquê disso, e passar por cada coluna de dados buscando dados estranhos e possíveis alterações que gostaríamos de fazer. 

**O segundo passo** então trata-se da remoção dos valores duplicados, que para esse dataset, eram poucos. 

**O terceiro passo** para limpeza completa de dados, é o preenchimento dos valores nulos, ou exclusão de alguma parte do dataframe onde o preenchimento de nulos não é possível.

### Categorização e Análise
Com os dados limpos, os próximos passos consistem em categorizar dados que possuem muitos valores únicos, e analisar os dados em busca de correlações com a métrica principal, que nesse caso era a coluna `debt`.

## Sobre as hipóteses iniciais e novas hipóteses

*Hipótese 1:* O **estado civil** dos clientes tem impacto sobre seu comportamento de pagamento de empréstimos.
Observamos que **sim**, o estado civil tem impacto sobre a incidência de inadimplência do cliente - a maior inadimplência está no grupo de não casados, enquanto a menor está com viúvas(os).

*Hipótese 2:* O **número de filhos** dos clientes tem impacto sobre seu comportamento de pagamento de empréstimos.
A quantidade de filhos não afeta de forma tão direta as taxas de inadimplência, **mas ter ou não filhos afeta**, com pessoas sem filhos com a menor taxa de inadimplência.

*Hipótese 3:* O nível de renda é um fator que influencia a inadimplência.
Temos diferentes taxas de inadimplência dependendo da renda, mas não parece que há uma relação linear entre maior renda - menor taxa de inadimplência. As menores taxas ficaram sim com aqueles com maior renda dentro do dataset, mas as piores taxas ficaram com quem tem uma renda média-alta, e a segunda menor taxa de inadimplência ficou com o grupo de baixa renda.

*Hipótese 4:* O propósito do empréstimo influencia na taxa de inadimplência.
Sim, isso se mostrou verdadeiro nos dados, com taxas de inadimplência mais baixas para aqueles que buscam empréstimo para lidar com atividades relacionadas a propriedades, e taxas mais altas para aqueles que buscam empréstimo para carros.

## Observações Gerais
A partir do exposto acima, é possível modelar um score de crédito que reflita as características do consumidor na hora da escolha de uma taxa de retorno (juros) sobre o empréstimo, levando em consideração o quão arriscado aquele empréstimo é. No geral, vemos que:

1. O cliente **mais arriscado** é aquele que **tem filhos**, **renda média-alta**, **solteiro/não casado**, e com **propósito de comprar um carro ou investir em educação**.

2. O cliente **menos arriscado** é aquele que **não tem filhos**, **renda alta**, **viúva(o)**, e com **propósito de comprar uma casa ou qualquer atividade relacionada ao mercado imobiliário**.
